In [ ]:
!pip install utilsforecast uni2ts

In [ ]:
!pip install --upgrade torch torchvision

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

from uni2ts.model.moirai2 import Moirai2Forecast, Moirai2Module

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

## Zero-shot forecasting

In [ ]:
HORIZON = 13

In [ ]:
# Create a Gluon dataframe


In [ ]:
# Initialize a Moirai model


In [ ]:
# Make predictions


In [ ]:
forecasts[0]

In [ ]:
# Function to transform raw predictions as a DataFrame
def create_fcsts_df(forecasts, multivariate=False):
    all_forecast_data = []
    for forecast_obj in forecasts:
        # Check if the forecast object has 'quantile' method (e.g., QuantileForecast)
        if hasattr(forecast_obj, 'quantile'):
            q10 = forecast_obj.quantile('0.1')
            q50 = forecast_obj.quantile('0.5')
            q90 = forecast_obj.quantile('0.9')
        # Otherwise, assume it's a SampleForecast and calculate quantiles from samples
        elif hasattr(forecast_obj, 'samples'):
            q10 = np.quantile(forecast_obj.samples, 0.1, axis=0)
            q50 = np.quantile(forecast_obj.samples, 0.5, axis=0)
            q90 = np.quantile(forecast_obj.samples, 0.9, axis=0)
        else:
            raise ValueError("Unsupported forecast object type. It must have either 'quantile' method or 'samples' attribute.")

        # Get start date and frequency

        if multivariate:
            # For multivariate forecasts, item_id is None, and samples are (prediction_length, num_series)
            # Get num_series and unique ids

            for i in range(num_series):
                # Set current id

                # Create datestamps

                # Create DataFrame

                # Append DataFrame

        else:
            # For univariate forecasts, item_id is present
            # Get id

            # Create dates

            # Create DataFrame

            # Append DataFrame

    # Concatenate all results

    return final_df

In [ ]:
# Parse forecasts as DataFrame

fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
# Create train and test template

# Define the cross-validation period

# Create predictor

# Run cross-validation


In [ ]:
len(cv_forecasts)

In [ ]:
cv_fcsts_df = create_fcsts_df(cv_forecasts)
cv_fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
# Merge actual values with cross-validation results

test_df.head()

In [ ]:
eval_df = evaluate(
    test_df,
    metrics=[mae, smape],
    models=['Moirai2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Forecasting with covariates

In [ ]:
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

In [ ]:
# Create Gluon dataframe with exogenous features


In [ ]:
train, test_template = split(exog_ds, offset=-(3*HORIZON))

test_data = test_template.generate_instances(
    prediction_length=HORIZON,
    windows=3,
    distance=HORIZON
)

# Moirai-2 does not support features, so use Moirai-1 or Moirai-MoE

# Create predictor

# Run cross-validation


In [ ]:
exog_cv_fcsts_df = create_fcsts_df(exog_cv_forecasts)
exog_cv_fcsts_df["Store"] = exog_cv_fcsts_df["Store"].astype(int)
exog_cv_fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=exog_cv_fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
exog_test_df = exog_cv_fcsts_df.merge(df[["Store", "Date", "Weekly_Sales"]], on=['Store', 'Date'], how='left')

eval_df = evaluate(
    exog_test_df,
    metrics=[mae, smape],
    models=['Moirai2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Multivariate forecasting

In [ ]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

In [ ]:
# Get number of unique ids

# Initialize the MultivariateGrouper


# Apply the MultivariateGrouper


In [ ]:
train, test_template = split(multivariate_ds, offset=-(3*HORIZON))

test_data = test_template.generate_instances(
    prediction_length=HORIZON,
    windows=3,
    distance=HORIZON
)

model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),
    prediction_length=HORIZON,
    context_length=len(df.query('Store == 1')),
    patch_size="auto",
    num_samples=100,
    # Adjust target_dim

    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
)

predictor = model.create_predictor(batch_size=32)
mv_cv_forecasts = predictor.predict(test_data.input)
mv_cv_forecasts = list(mv_cv_forecasts)

In [ ]:
mv_cv_fcsts_df = create_fcsts_df(mv_cv_forecasts, multivariate=True)
mv_cv_fcsts_df["Store"] = mv_cv_fcsts_df["Store"].astype(int)
mv_cv_fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=mv_cv_fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
mv_test_df = mv_cv_fcsts_df.merge(df[["Store", "Date", "Weekly_Sales"]], on=['Store', 'Date'], how='left')

eval_df = evaluate(
    mv_test_df,
    metrics=[mae, smape],
    models=['Moirai2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df